<a href="https://colab.research.google.com/github/Mukabony/Calculadora-Meta-Venda/blob/main/CHAT_IA_BR2T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CHAT IA BR2T**

##1. Primeira Célula - Instalação de Dependências 📦 ✅ (Validado)


In [238]:
!pip install langchain langchain-openai pandas langchain-community langchain_openai pydantic langgraph requests

##Célula 2 - Importação de Bibliotecas 📚 ✅ (Validado)


In [239]:
from langchain_openai import ChatOpenAI
import os
from google.colab import userdata
from typing import TypedDict, Annotated, Sequence, Dict, List
from pydantic import BaseModel
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
import requests
import json
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

##Célula 3 - Configuração de APIs e Modelos 🔑 ✅ (Validado)


In [240]:
# Configuração segura das chaves usando o sistema de secrets do Colab
os.environ['DEEPSEEK_API_KEY'] = userdata.get('DEEPSEEK_API_KEY')
os.environ['MILVUS_KEY'] = userdata.get('MILVUS_KEY')

# Definição da URL base da API Milvus
MILVUS_BASE_URL = "https://apiintegracao.milvus.com.br/api"

# Configuração do modelo DeepSeek (exemplo)
llm_deepseek = ChatOpenAI(
    model="deepseek-coder",
    temperature=0,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com/v1"
)

# Verificação de configuração
def verify_api_keys():
    required_keys = ['DEEPSEEK_API_KEY', 'MILVUS_KEY']
    missing_keys = [key for key in required_keys if not userdata.get(key)]
    if missing_keys:
        print("\n⚠ Atenção! As seguintes chaves estão faltando:")
        for key in missing_keys:
            print(f"- {key}")
        print("\nPor favor, adicione-as no gerenciador de secrets do Colab!")
        return False
    print("\n✅ Todas as chaves configuradas com sucesso!")
    return True

verify_api_keys()

llm = llm_deepseek



✅ Todas as chaves configuradas com sucesso!


##Célula 4 - Agente de Autenticação 🔐 ✅ (Validado)


In [241]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "Mensagens da conversa"]
    profile: Annotated[Dict, "Informações do perfil do usuário"]
    validation_result: Annotated[bool | None, "Resultado da validação"]
    validation_details: Annotated[Dict, "Detalhes da validação"]
    token_cliente: Annotated[str | None, "Token do cliente para uso posterior"]

class MilvusAPI:
    def __init__(self):
        self.base_url = MILVUS_BASE_URL
        self.headers = {
            "Authorization": os.getenv('MILVUS_KEY'),
            "Content-Type": "application/json"
        }

    def validate_tecnico(self, email: str) -> bool:
        """Valida técnico verificando se existe algum chamado associado"""
        url = f"{self.base_url}/chamado/listagem"
        payload = {
            "filtro_body": {
                "email_tecnico": email
            },
            "total_registros": 1
        }
        try:
            response = requests.post(url, headers=self.headers, json=payload)
            if response.status_code == 200:
                data = response.json()
                return len(data.get("lista", [])) > 0
            return False
        except Exception as e:
            print(f"Erro na validação do técnico: {e}")
            return False

    def validate_usuario(self, email: str) -> bool:
        """Valida usuário cliente"""
        url = f"{self.base_url}/usuario-cliente/listar"
        payload = {
            "filtro_body": {
                "email": email,
                "status": ""
            },
            "total_registros": 1,
            "pagina": 1
        }
        try:
            response = requests.post(url, headers=self.headers, json=payload)
            if response.status_code == 200:
                data = response.json()
                return len(data.get("lista", [])) > 0
            return False
        except Exception as e:
            print(f"Erro na validação do usuário: {e}")
            return False

    def validate_gestor(self, cnpj: str, state: AgentState) -> bool:
        """Valida gestor através do CNPJ da empresa e obtém o token do cliente"""
        url = f"{self.base_url}/cliente/busca"
        params = {"documento": cnpj}
        try:
            response = requests.get(url, headers=self.headers, params=params)
            if response.status_code == 200:
                data = response.json()
                if "lista" in data and len(data["lista"]) > 0:
                    state["token_cliente"] = data["lista"][0]["token"]
                    return True
            return False
        except Exception as e:
            print(f"Erro na validação do gestor: {e}")
            return False

    def listar_chamados(self, filtro_body: Dict) -> Dict:
        """Lista os chamados com base no filtro fornecido"""
        url = f"{self.base_url}/chamado/listagem"
        payload = {
            "filtro_body": filtro_body,
            "total_registros": 1000
        }
        try:
            response = requests.post(url, headers=self.headers, json=payload)
            if response.status_code == 200:
                return response.json()
            else:
                return {"erro": f"Erro na API: {response.status_code} - {response.text}"}
        except Exception as e:
            return {"erro": f"Exceção ao chamar a API: {str(e)}"}

# Instância da API Milvus
milvus_api = MilvusAPI()

# Perfis válidos com validação real
VALID_PROFILES = {
    "master": {
        "type": "master",
        "validation": lambda x, state: x.get("value") == "br2t"
    },
    "tecnico": {
        "type": "tecnico",
        "validation": lambda x, state: milvus_api.validate_tecnico(x.get("value"))
    },
    "usuario": {
        "type": "usuario",
        "validation": lambda x, state: milvus_api.validate_usuario(x.get("value"))
    },
    "gestor": {
        "type": "gestor",
        "validation": lambda x, state: milvus_api.validate_gestor(x.get("value"), state)
    }
}

def validate_profile(state: AgentState) -> AgentState:
    """Função de validação com integração real à API Milvus"""
    profile_data = state["profile"]
    profile_type = profile_data.get("profile")

    validation_details = {
        "profile_type": profile_type,
        "validation_steps": [],
        "errors": []
    }

    if profile_type not in VALID_PROFILES:
        validation_details["errors"].append(f"Tipo de perfil '{profile_type}' inválido")
        state["validation_result"] = False
        state["validation_details"] = validation_details
        return state

    profile_config = VALID_PROFILES[profile_type]
    try:
        validation_result = profile_config["validation"](profile_data, state)
        validation_details["validation_steps"].append({
            "step": "api_validation",
            "passed": validation_result,
            "profile": profile_type,
            "value": profile_data.get("value")
        })
        state["validation_result"] = validation_result
    except Exception as e:
        validation_details["errors"].append(f"Erro na validação: {str(e)}")
        state["validation_result"] = False

    state["validation_details"] = validation_details
    return state

# Configuração do grafo
workflow = StateGraph(AgentState)
workflow.add_node("validate", validate_profile)
workflow.set_entry_point("validate")
workflow.add_edge("validate", END)

# Compilação do grafo
validation_graph = workflow.compile()


##🗒️ Célula 5 - Dicionário da Documentação da API


In [242]:
api_documentation = """
listagemChamados - listagemChamados
Listagem de chamados
POST https://apiintegracao.milvus.com.br/api/chamado/listagem
Headers:
 - Authorization: Obrigatório o uso do token de autenticação.
Parâmetros:
 - is_descending (boolean, opcional)
 - order_by (string, opcional)
 - total_registros (integer, opcional)
 - pagina (integer, opcional)

Request Body Exemplo:

  "filtro_body":
    "categoria_primaria": "SANKHYA",
    "categoria_secundaria": "CONTROLAR OS OPERACIONAL SEM CUSTO",
    "email_tecnico": "rjoins@br2t.com.br",
    ...



Opções de filtro 'status':
 1 ou "AgAtendimento"
 2 ou "Atendendo"
 3 ou "Pausado"
 4 ou "Finalizado"
 5 ou "Conferencia"
 6 ou "Agendado"
 7 ou "Expirado"
 9 ou "ChamadosAbertos"
10 ou "Todos"
11 ou "AgSolucao"
13 ou "AbertosNaoAgendados"
14 ou "SemTecnico"
"""


##**Célula 6 - Novo Dicionário de Perguntas Pré-Definidas por Perfil**

In [243]:
perguntas_padrao_profile = {
    "master": [
        {
            "pergunta": "Quantos chamados foram abertos na última semana, separados por cada cliente?",
            "filtro_body": {
                "status": "Todos",
                "data_inicio": "2025-01-03",
                "data_fim": "2025-01-10"
            }
        },
        {
            "pergunta": "Quais clientes têm mais chamados críticos em aberto?",
            "filtro_body": {
                "status": "ChamadosAbertos",
                "prioridade": "Crítico"
            }
        },
        {
            "pergunta": "Gostaria de ver a lista de chamados atrasados (SLA estourado) para todos os clientes.",
            "filtro_body": {
                "status": "Todos",
                "sla_resposta": "Estourado"
            }
        },
        {
            "pergunta": "Quero um relatório de quantos chamados cada técnico fechou neste mês, por empresa.",
            "filtro_body": {
                "status": "Finalizado",
                "data_inicio": "2025-01-01",
                "data_fim": "2025-01-31"
            }
        },
        {
            "pergunta": "Liste todos os chamados com prioridade média e alto impacto, preciso analisar a urgência do suporte.",
            "filtro_body": {
                "status": "Todos",
                "prioridade": "Médio",
                "impacto": "Alto"
            }
        }
    ],
    "usuario": [
        {
            "pergunta": "Quais chamados eu tenho em aberto agora?",
            "filtro_body": {
                "status": "ChamadosAbertos"
            }
        },
        {
            "pergunta": "Mostre meus chamados finalizados nos últimos 30 dias, por favor.",
            "filtro_body": {
                "status": "Finalizado",
                "data_inicio": "2025-01-10",
                "data_fim": "2025-02-09"
            }
        },
        {
            "pergunta": "Preciso ver meu histórico de chamados sobre erro paytrack e configurações, para comparar.",
            "filtro_body": {
                "status": "Todos",
                "assunto": "backup"
            }
        },
        {
            "pergunta": "Quero saber quais chamados estão com SLA estourado, pois ainda não fui atendido.",
            "filtro_body": {
                "status": "A fazer",
                "sla_resposta": "Estourado"
            }
        },
        {
            "pergunta": "Liste meus chamados com prioridade alta e me informe se já foram respondidos.",
            "filtro_body": {
                "status": "Todos",
                "prioridade": "Alto"
            }
        }
    ],
    "gestor": [
        {
            "pergunta": "Quantos chamados meus usuários abriram este mês?",
            "filtro_body": {
                "status": "Todos",
                "data_inicio": "2025-01-01",
                "data_fim": "2025-01-31"
            }
        },
        {
            "pergunta": "Liste todos os chamados críticos que estão em aberto na minha empresa.",
            "filtro_body": {
                "status": "ChamadosAbertos",
                "prioridade": "Crítico"
            }
        },
        {
            "pergunta": "Quero verificar chamados finalizados sobre backup para saber se está tudo certo.",
            "filtro_body": {
                "status": "Finalizado",
                "assunto": "backup"
            }
        },
        {
            "pergunta": "Quais chamados estão com SLA de resposta estourado, preciso acompanhar.",
            "filtro_body": {
                "status": "Todos",
                "sla_resposta": "Estourado"
            }
        },
        {
            "pergunta": "Me mostre todos os chamados preventivos abertos hoje na minha empresa.",
            "filtro_body": {
                "status": "A fazer",
                "tipo_ticket": "Preventivo",
                "data_inicio": "2025-01-10",
                "data_fim": "2025-01-10"
            }
        }
    ],
    "tecnico": [
        {
            "pergunta": "Quais chamados estão atribuídos a mim e ainda não foram solucionados?",
            "filtro_body": {
                "status": "A fazer"
            }
        },
        {
            "pergunta": "Preciso ver meus chamados finalizados neste mês, para acompanhar minhas horas.",
            "filtro_body": {
                "status": "Finalizado",
                "data_inicio": "2025-01-01",
                "data_fim": "2025-01-31"
            }
        },
        {
            "pergunta": "Qual a lista de chamados preventivos que estou responsável hoje?",
            "filtro_body": {
                "status": "A fazer",
                "tipo_ticket": "Preventivo",
                "data_inicio": "2025-01-10",
                "data_fim": "2025-01-10"
            }
        },
        {
            "pergunta": "Quero ver todos os chamados de backup que fechei esta semana, para validar se resolvi corretamente.",
            "filtro_body": {
                "status": "Finalizado",
                "assunto": "backup",
                "data_inicio": "2025-01-08",
                "data_fim": "2025-01-10"
            }
        },
        {
            "pergunta": "Liste os chamados atrasados em SLA que estão sob minha responsabilidade.",
            "filtro_body": {
                "status": "Todos",
                "sla_resposta": "Estourado"
            }
        }
    ]
}


##**Célula 7 - Função buscar_pergunta_predefinida**

In [244]:
def buscar_pergunta_predefinida(pergunta_usuario: str, profile_type: str) -> dict:
    """
    Verifica se a pergunta do usuário está no dicionário de perguntas pré-definidas
    para o perfil informado. Se encontrar, retorna {'status': 'found', 'filtro_body': {...}}.
    Caso contrário, retorna {'status': 'not_found'}.
    """
    perguntas_profile = perguntas_padrao_profile.get(profile_type, [])

    # Simples comparação exata (poderia ser fuzzy, caso queira expandir)
    for item in perguntas_profile:
        if item["pergunta"].strip().lower() == pergunta_usuario.strip().lower():
            return {
                "status": "found",
                "filtro_body": item["filtro_body"]
            }

    return {"status": "not_found"}


##**Célula 8 - Ferramenta de Validação Rápida**


In [245]:
def validacao_rapida_chamados(filtro_body: Dict) -> Dict:
    """
    Faz uma validação rápida (Tool) para verificar se existem registros.
    Envia total_registros=1 para a API, retornando True/False ou erro.
    Retorna:
      {
        "status": "ok",
        "tem_registros": True/False,
        "erro": None (ou string de erro)
      }
    """
    url = f"{MILVUS_BASE_URL}/chamado/listagem"
    payload = {
        "filtro_body": filtro_body,
        "total_registros": 1
    }
    headers = {
        "Authorization": os.getenv('MILVUS_KEY'),
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code == 200:
            data = response.json()
            total = data.get("meta", {}).get("paginate", {}).get("total", 0)
            tem_registros = (total > 0)
            return {"status": "ok", "tem_registros": tem_registros, "erro": None}
        else:
            return {"status": "erro", "tem_registros": False, "erro": f"API Error {response.status_code}: {response.text}"}
    except Exception as e:
        return {"status": "erro", "tem_registros": False, "erro": str(e)}


##**Célula 9 - Ferramentas e Funções Auxiliares**

In [246]:
def exibir_sugestoes_perfil(profile_type: str, pergunta_falada: str) -> str:
    """
    Monta uma mensagem amigável com as perguntas disponíveis para aquele perfil,
    excluindo a pergunta que falhou (se for encontrada).
    Retorna um texto em formato de string para exibição ao usuário.
    """
    perguntas_disponiveis = []
    for item in perguntas_padrao_profile.get(profile_type, []):
        # Se for diferente da pergunta que acabou de falhar, lista
        if item["pergunta"].strip().lower() != pergunta_falada.strip().lower():
            perguntas_disponiveis.append(item["pergunta"])

    if not perguntas_disponiveis:
        return (
            "Não encontrei nenhum chamado para essa consulta e não há perguntas adicionais disponíveis.\n"
            "Por favor, tente reformular sua pergunta ou entre em contato com o suporte."
        )

    texto = (
        "Não encontrei nenhum registro para essa consulta ou ocorreu um erro.\n"
        "Estou em fase de desenvolvimento pela RJOINS.\n\n"
        "Você pode tentar uma destas perguntas (digite o número ou a própria pergunta):\n"
    )
    for i, pergunta in enumerate(perguntas_disponiveis, start=1):
        texto += f"{i}) {pergunta}\n"
    return texto

def completar_filtro_por_perfil(filtro_body: Dict, profile_data: Dict, token_cliente: str | None) -> Dict:
    """
    Insere campos de perfil (email_conferencia, email_tecnico, cliente_token) de acordo com o tipo do perfil.
    Retorna uma cópia atualizada do filtro_body.
    """
    profile_type = profile_data.get("profile")
    valor = profile_data.get("value")

    # Copiamos para evitar mutações indevidas
    novo_filtro = dict(filtro_body)

    if profile_type == "usuario":
        novo_filtro["email_conferencia"] = valor
    elif profile_type == "tecnico":
        novo_filtro["email_tecnico"] = valor
    elif profile_type == "gestor":
        if token_cliente:
            novo_filtro["cliente_token"] = token_cliente
        else:
            # Se não houver token, não conseguimos filtrar
            pass
    # master não precisa de nada adicional

    return novo_filtro


##🤖 Célula 6 - Agente de Identificação de Intenção (Router Agent) ✅ (validado)

In [247]:
def identificar_intencao(state: dict) -> dict:
    """Usa o LLM para identificar a intenção da pergunta."""
    ultima_mensagem = state["messages"][-1]
    pergunta = ultima_mensagem.content if isinstance(ultima_mensagem, HumanMessage) else ""

    prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente especializado em identificar intenções de perguntas.
        Identifique a intenção da pergunta do usuário e responda apenas com uma das seguintes palavras-chave:
        - listar_chamados
        - outra_intencao"""),
        ("human", "{pergunta}")
    ])
    chain = prompt | llm
    resposta = chain.invoke({"pergunta": pergunta})

    state["intencao"] = resposta.content.strip()
    return state

##🧠 Célula 7 - Agente LLM para Decidir Parâmetros da API


In [248]:
def agente_decide_filtros(state: dict) -> dict:
    """Usa o LLM para decidir quais campos utilizar na chamada da API."""
    ultima_mensagem = state["messages"][-1]
    pergunta = ultima_mensagem.content if isinstance(ultima_mensagem, HumanMessage) else ""

    prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente especializado em extrair parâmetros de filtro para uma API de chamados.
        Sua função é analisar a pergunta do usuário e retornar APENAS um JSON com o campo 'filtro_body' contendo os filtros necessários.

        Regras importantes:
        1. Use apenas campos válidos:
           - status: para estado do chamado
           - assunto: para busca textual
           - prioridade: para nível de urgência
           - data_inicio e data_fim: para filtros de período
           - tipo_ticket: para tipo específico de chamado
           - sla_resposta: para status do SLA
           - impacto: para nível de impacto

        2. Valores válidos para status:
           - "AgAtendimento" - aguardando atendimento
           - "Atendendo" - em atendimento
           - "Pausado" - pausado
           - "Finalizado" - finalizado
           - "ChamadosAbertos" - todos abertos
           - "Todos" - todos os status

        3. Se não houver menção específica:
           - Use "status": "Todos"
           - Evite adicionar campos desnecessários

        4. Para buscas textuais:
           - Use o campo "assunto" com o termo mencionado

        Exemplos de respostas:
        Para "chamados sobre Sankhya":
        {{"filtro_body": {{"status": "Todos", "assunto": "sankhya"}}}}

        Para "chamados críticos abertos":
        {{"filtro_body": {{"status": "ChamadosAbertos", "prioridade": "Crítico"}}}}

        Para "chamados do último mês":
        {{"filtro_body": {{"status": "Todos", "data_inicio": "2025-01-01", "data_fim": "2025-01-31"}}}}
        """),
        ("human", "{pergunta}")
    ])

    # Invoca o LLM
    chain = prompt | llm
    resposta = chain.invoke({"pergunta": pergunta})

    # Processa a resposta
    resposta_texto = resposta.content.strip()
    resposta_texto = resposta_texto.replace("```json", "").replace("```", "").strip()

    try:
        # Tenta fazer o parse do JSON
        filtros = json.loads(resposta_texto)

        # Validações e correções
        if not isinstance(filtros, dict):
            filtros = {"filtro_body": {}}
        elif "filtro_body" not in filtros:
            filtros = {"filtro_body": filtros}

        # Garante que filtro_body existe e é um dicionário
        if not isinstance(filtros["filtro_body"], dict):
            filtros["filtro_body"] = {}

        # Garante que status seja válido
        status_validos = [
            "AgAtendimento", "Atendendo", "Pausado", "Finalizado",
            "ChamadosAbertos", "Todos"
        ]
        current_status = filtros["filtro_body"].get("status")
        if not current_status or current_status not in status_validos:
            filtros["filtro_body"]["status"] = "Todos"

        # Remove campos vazios ou None
        filtros["filtro_body"] = {
            k: v for k, v in filtros["filtro_body"].items()
            if v is not None and v != ""
        }

        state["filtros_api"] = filtros

    except json.JSONDecodeError as e:
        # Em caso de erro no JSON, usa um filtro padrão
        state["filtros_api"] = {"filtro_body": {"status": "Todos"}}
        state["erro"] = f"Erro ao processar filtros da API: {str(e)}"

    # Debug log
    print("\n[DEBUG] Pergunta:", pergunta)
    print("[DEBUG] Filtros gerados:", json.dumps(state["filtros_api"], indent=2))

    return state

In [249]:
def testar_agente_filtros():
    """
    Função para testar isoladamente o agente_decide_filtros com diferentes cenários
    """
    # Lista de perguntas para teste
    perguntas_teste = [
        "Mostre todos os chamados sobre Sankhya",
        "Quais chamados estão abertos hoje?",
        "Preciso ver os chamados críticos",
        "Liste os chamados do último mês",
        "Chamados com erro de backup",
        "Quero ver chamados atrasados",
        "Mostre chamados de alta prioridade em atendimento",
        "Chamados pausados sobre erro no sistema",
        "Lista de preventivas desta semana",
        "Chamados com SLA estourado"
    ]

    print("\n=== INÍCIO DOS TESTES DO AGENTE DE FILTROS ===\n")

    for i, pergunta in enumerate(perguntas_teste, 1):
        print(f"\n--- Teste #{i} ---")
        print(f"Pergunta: {pergunta}")

        # Cria um estado inicial para o teste
        estado_teste = {
            "messages": [HumanMessage(content=pergunta)],
            "profile": {"profile": "tecnico", "value": "teste@exemplo.com"}
        }

        try:
            # Executa o agente
            resultado = agente_decide_filtros(estado_teste)

            # Exibe o resultado formatado
            if "filtros_api" in resultado:
                print("\nFiltros gerados:")
                print(json.dumps(resultado["filtros_api"], indent=2))

            if "erro" in resultado:
                print("\nErro encontrado:", resultado["erro"])

        except Exception as e:
            print(f"\nERRO NA EXECUÇÃO: {str(e)}")

        print("\n" + "-"*50)

    print("\n=== FIM DOS TESTES ===")

# Executar os testes
testar_agente_filtros()


=== INÍCIO DOS TESTES DO AGENTE DE FILTROS ===


--- Teste #1 ---
Pergunta: Mostre todos os chamados sobre Sankhya

[DEBUG] Pergunta: Mostre todos os chamados sobre Sankhya
[DEBUG] Filtros gerados: {
  "filtro_body": {
    "status": "Todos",
    "assunto": "Sankhya"
  }
}

Filtros gerados:
{
  "filtro_body": {
    "status": "Todos",
    "assunto": "Sankhya"
  }
}

--------------------------------------------------

--- Teste #2 ---
Pergunta: Quais chamados estão abertos hoje?

[DEBUG] Pergunta: Quais chamados estão abertos hoje?
[DEBUG] Filtros gerados: {
  "filtro_body": {
    "status": "ChamadosAbertos"
  }
}

Filtros gerados:
{
  "filtro_body": {
    "status": "ChamadosAbertos"
  }
}

--------------------------------------------------

--- Teste #3 ---
Pergunta: Preciso ver os chamados críticos

[DEBUG] Pergunta: Preciso ver os chamados críticos
[DEBUG] Filtros gerados: {
  "filtro_body": {
    "status": "Todos",
    "prioridade": "Cr\u00edtico"
  }
}

Filtros gerados:
{
  "filtro_

##🛠️ Célula 8 - Implementação da Ferramenta ChamadosAPITool


In [250]:
def obter_chamados(state: dict) -> dict:
    """Obtém os chamados usando a MilvusAPI."""
    filtros_api = state.get("filtros_api", {})
    profile_data = state["profile"]
    profile_type = profile_data.get("profile")

    # Caso a LLM retorne "filtro_body" dentro do JSON, pegamos só esse dicionário
    if "filtro_body" in filtros_api:
        filtro_body = filtros_api["filtro_body"]
    else:
        filtro_body = filtros_api  # fallback

    # Aplica dados de perfil
    token_cliente = state.get("token_cliente")
    filtro_completo = completar_filtro_por_perfil(filtro_body, profile_data, token_cliente)

    # LOG de debug
    print("\n[DEBUG] Filtro final para obter_chamados:", filtro_completo)

    # Chama a API real
    dados_chamados = milvus_api.listar_chamados(filtro_completo)
    print("[DEBUG] Retorno da API:", dados_chamados, "\n")

    if "erro" in dados_chamados:
        state["erro"] = dados_chamados["erro"]
    else:
        state["dados_chamados"] = dados_chamados
    return state


##**Interpretar a Escolha**

In [251]:
def interpretar_opcao_sugerida(state: dict) -> dict:
    suggestions = state.get("suggestions", [])
    ultima_mensagem = ""
    if state["messages"]:
        msg = state["messages"][-1]
        if isinstance(msg, HumanMessage):
            ultima_mensagem = msg.content.strip()

    # Corrigido o prompt para usar chaves duplas onde necessário
    prompt_llm = ChatPromptTemplate.from_messages([
        ("system",
         """Você é um assistente que decide se a mensagem do usuário corresponde a escolher
         uma opção da lista ou se é uma pergunta nova.
         Responda apenas em JSON, no formato:
         {{
           "status": "selected" ou "new_question",
           "index": <numero ou null>,
           "text": "<texto inteiro>"
         }}"""),  # Note as chaves duplas aqui
        ("human",
         "Opções disponíveis: {options}\nUsuário disse: {message}")
    ])

    # Invoca a LLM com as variáveis corretas
    chain = prompt_llm | llm
    resposta_llm = chain.invoke({
        "options": json.dumps(suggestions, ensure_ascii=False),
        "message": ultima_mensagem
    })

    try:
        dados = json.loads(resposta_llm.content.strip())
    except:
        dados = {
            "status": "new_question",
            "index": None,
            "text": ultima_mensagem
        }

    state["escolha_sugestao"] = dados
    return state

##🧮 Célula 9 - Funções Auxiliares para Processamento de Dados


In [252]:
def formatar_resposta(state: dict) -> dict:
    dados_chamados = state.get("dados_chamados", {})
    chamados = dados_chamados.get("lista", [])
    if not chamados:
        state["resposta"] = "Não foram encontrados chamados para os critérios solicitados."
        return state

    resposta = "### Chamados Encontrados\n\n"
    resposta += "| Código | Assunto | Data de Criação | Status |\n"
    resposta += "|--------|---------|-----------------|--------|\n"
    for chamado in chamados:
        resposta += (
            f"| {chamado.get('codigo', 'N/A')} "
            f"| {chamado.get('assunto', 'N/A')} "
            f"| {chamado.get('data_criacao', 'N/A')} "
            f"| {chamado.get('status', 'N/A')} |\n"
        )
    state["resposta"] = resposta
    return state


##🔄 Célula 10 - Configuração do Grafo Principal


In [253]:
# =========================================================
# Célula 10 - Configuração do Grafo Principal (AJUSTADA)
# =========================================================

from typing import TypedDict, Annotated, Sequence, Dict, Union, Tuple, List
from langgraph.graph import StateGraph, END

class ExtendedAgentState(TypedDict, total=False):
    messages: Annotated[Sequence[BaseMessage], "Mensagens da conversa"]
    profile: Annotated[Dict, "Informações do perfil do usuário"]
    validation_result: Annotated[bool | None, "Resultado da validação"]
    validation_details: Annotated[Dict, "Detalhes da validação"]
    token_cliente: Annotated[str | None, "Token do cliente para uso posterior"]
    intencao: Annotated[str | None, "Intenção da pergunta"]
    filtros_api: Annotated[Dict | None, "Filtros para a API"]
    dados_chamados: Annotated[Dict | None, "Dados dos chamados"]
    resposta: Annotated[str | None, "Resposta final ao usuário"]
    erro: Annotated[str | None, "Mensagem de erro"]

# ---------------------------------
# 1) Nó: validate_profile (igual antes)
# 2) Nó: buscar_predefinida (novo)
# 3) Nó: identificar_intencao (só se predefinida não encontrada)
# 4) Nó: agente_decide_filtros (ou se foi dicionário, pula)
# 5) Nó: validacao_rapida
# 6) Nó: se tem registro => obter_chamados => formatar_resposta => END
# 7) Nó: se não tem registro ou erro => sugerir_opcoes => END (AJUSTE AQUI)
#    (ou se quiser re-perguntar, tratar de outra forma)
# ---------------------------------

def node_buscar_predefinida(state: ExtendedAgentState) -> ExtendedAgentState:
    """
    Verifica se a pergunta do usuário (última mensagem) está no dicionário de perguntas
    pré-definidas para o perfil.
    Se encontrar, salva em state["filtros_api"] e segue para validação rápida.
    Senão, segue para identificar_intencao.
    """
    pergunta = ""
    if state["messages"]:
        msg = state["messages"][-1]
        if isinstance(msg, HumanMessage):
            pergunta = msg.content
    profile_type = state["profile"].get("profile", "")
    resultado = buscar_pergunta_predefinida(pergunta, profile_type)
    if resultado["status"] == "found":
        state["filtros_api"] = {"filtro_body": resultado["filtro_body"]}
        # Usaremos a rota para "validacao_rapida"
        state["intencao"] = "listar_chamados"  # para padronizar o fluxo
    else:
        # Não encontrado => iremos para identificar_intencao
        state["intencao"] = None
    return state

def node_validacao_rapida(state: ExtendedAgentState) -> ExtendedAgentState:
    """
    Se state["filtros_api"] existir, faz uma checagem de 1 registro.
    Se tem registro => prossegue para obter_chamados
    Se não tem => gera sugestão e "erro" = 'no_data'
    """
    if not state.get("filtros_api"):
        # Se não há filtro, não há o que validar
        return state

    # Recupera e completa o filtro
    profile_data = state["profile"]
    filtro_body = state["filtros_api"].get("filtro_body", {})
    token_cliente = state.get("token_cliente")
    filtro_completo = completar_filtro_por_perfil(filtro_body, profile_data, token_cliente)

    resultado = validacao_rapida_chamados(filtro_completo)
    if resultado["status"] == "ok":
        if resultado["tem_registros"]:
            # OK, segue fluxo
            pass
        else:
            # Tem status ok mas total=0 => sem registros
            state["erro"] = "no_data"
    else:
        # status=erro => erro de API
        state["erro"] = f"Validação rápida falhou: {resultado['erro']}"
    return state

def node_sugerir_opcoes(state: ExtendedAgentState) -> ExtendedAgentState:
    """
    Gera a mensagem de sugestão de perguntas do dicionário para o perfil,
    e salva a lista em state["suggestions"] para o próximo passo.
    """
    pergunta_falada = ""
    if state["messages"]:
        msg = state["messages"][-1]
        if isinstance(msg, HumanMessage):
            pergunta_falada = msg.content.strip()

    profile_type = state["profile"].get("profile", "")
    perguntas_completas = perguntas_padrao_profile.get(profile_type, [])

    perguntas_disponiveis = []
    for item in perguntas_completas:
        if item["pergunta"].strip().lower() != pergunta_falada.lower():
            perguntas_disponiveis.append(item["pergunta"])

    state["suggestions"] = perguntas_disponiveis

    if not perguntas_disponiveis:
        texto = (
            "Não encontrei nenhum chamado para essa consulta e não há perguntas adicionais disponíveis.\n"
            "Por favor, tente reformular sua pergunta ou entre em contato com o suporte."
        )
        state["resposta"] = texto
        return state

    texto = (
        "Não encontrei nenhum registro para essa consulta ou ocorreu um erro.\n"
        "Estou em fase de desenvolvimento pela RJOINS.\n\n"
        "Você pode tentar uma destas perguntas (digite o número ou a própria pergunta):\n"
    )
    for i, perg_sug in enumerate(perguntas_disponiveis, start=1):
        texto += f"{i}) {perg_sug}\n"

    state["resposta"] = texto
    return state

def node_interpretar_opcao_sugerida(state: ExtendedAgentState) -> ExtendedAgentState:
    """
    Chama a função interpretar_opcao_sugerida e,
    se for 'selected', substitui a última mensagem do usuário pela pergunta real.
    """
    state = interpretar_opcao_sugerida(state)
    escolha = state.get("escolha_sugestao", {})
    if escolha.get("status") == "selected":
        idx = escolha.get("index") or 1
        suggestions = state.get("suggestions", [])
        if 1 <= idx <= len(suggestions):
            pergunta_escolhida = suggestions[idx - 1]
            if state["messages"] and isinstance(state["messages"][-1], HumanMessage):
                state["messages"][-1] = HumanMessage(content=pergunta_escolhida)
    return state

def route_after_interpretar_opcao(state: ExtendedAgentState) -> str:
    """
    Se o usuário escolheu uma opção, vamos para 'buscar_predefinida'.
    Senão, se for 'new_question', vamos para 'identificar_intencao'.
    """
    escolha = state.get("escolha_sugestao", {})
    if escolha.get("status") == "selected":
        return "buscar_predefinida"
    else:
        return "identificar_intencao"

# Montagem do grafo principal
main_workflow = StateGraph(ExtendedAgentState)

# Adicionando nós
main_workflow.add_node("validate_profile", validate_profile)       # (1)
main_workflow.add_node("buscar_predefinida", node_buscar_predefinida)  # (2)
main_workflow.add_node("identificar_intencao", identificar_intencao)   # (3)
main_workflow.add_node("decidir_filtros", agente_decide_filtros)       # (4)
main_workflow.add_node("validacao_rapida", node_validacao_rapida)      # (5)
main_workflow.add_node("obter_chamados", obter_chamados)               # (6)
main_workflow.add_node("formatar_resposta", formatar_resposta)
main_workflow.add_node("sugerir_opcoes", node_sugerir_opcoes)

# Entry point
main_workflow.set_entry_point("validate_profile")

# 1) validate_profile -> se validation_result True => buscar_predefinida, senão END
def route_after_validate(state: ExtendedAgentState) -> str:
    if state.get("validation_result"):
        return "buscar_predefinida"
    else:
        return END

main_workflow.add_conditional_edges("validate_profile", route_after_validate)

# 2) buscar_predefinida -> se intencao= "listar_chamados" => validacao_rapida, senão identificar_intencao
def route_after_buscar_predefinida(state: ExtendedAgentState) -> str:
    if state.get("intencao") == "listar_chamados":
        return "validacao_rapida"
    else:
        return "identificar_intencao"

main_workflow.add_conditional_edges("buscar_predefinida", route_after_buscar_predefinida)

# 3) identificar_intencao -> se "listar_chamados" => decidir_filtros, else END
def route_after_intencao(state: ExtendedAgentState) -> str:
    if state.get("intencao") == "listar_chamados":
        return "decidir_filtros"
    else:
        return END

main_workflow.add_conditional_edges("identificar_intencao", route_after_intencao)

# 4) decidir_filtros -> validacao_rapida
main_workflow.add_edge("decidir_filtros", "validacao_rapida")

# 5) validacao_rapida -> se erro is None => obter_chamados, senão sugerir_opcoes
def route_after_validacao_rapida(state: ExtendedAgentState) -> str:
    if state.get("erro"):
        return "sugerir_opcoes"
    else:
        return "obter_chamados"

main_workflow.add_conditional_edges("validacao_rapida", route_after_validacao_rapida)

# 6) obter_chamados -> formatar_resposta
main_workflow.add_edge("obter_chamados", "formatar_resposta")

# 7) formatar_resposta -> END
main_workflow.add_edge("formatar_resposta", END)

# ------------------------------------
# AJUSTE AQUI PARA EVITAR LOOP INFINITO
# ------------------------------------
# Antes: main_workflow.add_edge("sugerir_opcoes", "interpretar_opcao_sugerida")
# Substituímos pela linha abaixo para encerrar o fluxo diretamente:
main_workflow.add_edge("sugerir_opcoes", END)
# ------------------------------------

# Se você quiser manter a lógica de interpretar escolha, basta comentar a linha acima,
# reativar a antiga e ter algum critério adicional para não cair no loop.

main_workflow.add_node("interpretar_opcao_sugerida", node_interpretar_opcao_sugerida)
main_workflow.add_conditional_edges("interpretar_opcao_sugerida", route_after_interpretar_opcao)

# Compila o grafo final
agent_graph = main_workflow.compile()


##📝 Célula 11 - Fluxo Principal de Execução


In [254]:
def fluxo_principal(profile_usuario, pergunta_usuario):
    try:
        # Configuração inicial do estado
        initial_state = {
            "messages": [HumanMessage(content=pergunta_usuario)],
            "profile": profile_usuario,
            "validation_result": None,
            "validation_details": {},
            "token_cliente": None
        }

        # Executa o grafo com o estado inicial
        result = agent_graph.invoke(initial_state)

        # Exibe a resposta formatada
        if "resposta" in result:
            print(result["resposta"])
        elif "erro" in result:
            print(f"Erro: {result['erro']}")
        else:
            print("Nenhuma resposta ou erro encontrado.")

    except Exception as e:
        print(f"Erro na execução: {e}")

##🧪 Célula 12 - Testando o Fluxo com Diferentes Perfis


In [255]:
# Teste 1: Pergunta pré-definida do usuário
profile_usuario = {
    "profile": "usuario",
    "value": "lais.castro@pailon.com.br"
}
pergunta_usuario = "Quais chamados com assunto Lobolo?"

print("=== Teste 1: Pergunta pré-definida do usuário ===")
fluxo_principal(profile_usuario, pergunta_usuario)

=== Teste 1: Pergunta pré-definida do usuário ===

[DEBUG] Pergunta: Quais chamados com assunto Lobolo?
[DEBUG] Filtros gerados: {
  "filtro_body": {
    "status": "Todos",
    "assunto": "Lobolo"
  }
}
Não encontrei nenhum registro para essa consulta ou ocorreu um erro.
Estou em fase de desenvolvimento pela RJOINS.

Você pode tentar uma destas perguntas (digite o número ou a própria pergunta):
1) Quais chamados eu tenho em aberto agora?
2) Mostre meus chamados finalizados nos últimos 30 dias, por favor.
3) Preciso ver meu histórico de chamados sobre erro paytrack e configurações, para comparar.
4) Quero saber quais chamados estão com SLA estourado, pois ainda não fui atendido.
5) Liste meus chamados com prioridade alta e me informe se já foram respondidos.



In [256]:
# =========================================
# CÉLULA NOVA: Loop de Conversa
# =========================================
def iniciar_conversa(profile_usuario: dict):
    """
    Exemplo de loop que mantém o estado da conversa.
    O usuário digita repetidamente e podemos seguir com as sugestões.
    """
    # Montamos o estado inicial
    state: ExtendedAgentState = {
        "messages": [],
        "profile": profile_usuario,
        "validation_result": None,
        "validation_details": {},
        "token_cliente": None
    }

    print("Digite 'sair' para encerrar.\n")
    while True:
        user_input = input("Você: ")
        if not user_input or user_input.strip().lower() == "sair":
            print("Encerrando conversa.")
            break

        # Adiciona a nova mensagem do usuário
        state["messages"].append(HumanMessage(content=user_input))

        # Processa no grafo
        state = agent_graph.invoke(state)

        # Exibe resposta do assistente
        resposta = state.get("resposta", "")
        if resposta:
            print(f"Assistente: {resposta}")
        elif state.get("erro"):
            print(f"Assistente (ERRO): {state['erro']}")
        else:
            print("Assistente: (sem resposta)")

        # Se quiser checar se finalizamos o grafo:
        # if state.get("__graph_state__") == "END":
        #     print("Fluxo chegou ao END. Podemos continuar ou encerrar.\n")
        #     # break


In [257]:
profile_usuario = {"profile": "usuario", "value": "lais.castro@pailon.com.br"}
iniciar_conversa(profile_usuario)

Digite 'sair' para encerrar.

Você: Quantos chamados tenho com asssunto backup?

[DEBUG] Pergunta: Quantos chamados tenho com asssunto backup?
[DEBUG] Filtros gerados: {
  "filtro_body": {
    "status": "Todos",
    "assunto": "backup"
  }
}
Assistente: Não encontrei nenhum registro para essa consulta ou ocorreu um erro.
Estou em fase de desenvolvimento pela RJOINS.

Você pode tentar uma destas perguntas (digite o número ou a própria pergunta):
1) Quais chamados eu tenho em aberto agora?
2) Mostre meus chamados finalizados nos últimos 30 dias, por favor.
3) Preciso ver meu histórico de chamados sobre erro paytrack e configurações, para comparar.
4) Quero saber quais chamados estão com SLA estourado, pois ainda não fui atendido.
5) Liste meus chamados com prioridade alta e me informe se já foram respondidos.

Você: 1
Assistente: Não encontrei nenhum registro para essa consulta ou ocorreu um erro.
Estou em fase de desenvolvimento pela RJOINS.

Você pode tentar uma destas perguntas (digit

KeyboardInterrupt: Interrupted by user